In [6]:
# Pytorch Imports
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.data
import pandas as pd
import datetime
import time

In [7]:
# Function to generate two csv files for training/testing
def generate_train_test_data(file_name, percent_test=0.8):
    ''' 
        Will divide a dataset into both test and training sets
        
        Args:
        
        file_name (string): path to the csv file containing all data
        percent_test (float): percentage of dataset to set aside for testing
        
        returns nothing
    '''
    # Open file and place into dataframe
    columns = ['date', 'open', 'high', 'low', 'close', 'volume', '50ma', 'label']
    stock_data = pd.read_csv(file_name)[columns]
    stock_data['date'] = stock_data['date'].apply(lambda x: time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d").timetuple()), convert_dtype=True)
    stock_data = stock_data.dropna()
    return stock_data

In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    # total number of batches we can make
    n_batches = len(arr)//batch_size

    # Keep only enough days to make full batches
    arr = arr[:n_batches * batch_size]

    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:batch_size, n:n+seq_length]
        # The targets
        y = arr[:batch_size, -1:]
        yield x, y

In [9]:
def one_hot_encode(arr, n_labels):

    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [10]:
train, test = DataLoader(arr, .1)
num_classes = 3
try:
    for i, batch in enumerate(train):
        inputs = batch[:,:-1].float()
        inputs.unsqueeze_(1)   
        targets = batch[:,-1].int()
        targets.unsqueeze_(1)  
        targets = one_hot_encode(targets, 3)

except IndexError:
    pass


NameError: name 'arr' is not defined

In [11]:
print('x\n', inputs)
print('\ny\n', targets)

NameError: name 'inputs' is not defined

In [12]:
def DataLoader(arr, val_frac):
    #Create train and test splits based on val_frac

    test_end_idx = int(arr.shape[0] * val_frac)
    train_data = arr[test_end_idx+1:]
    test_data = arr[0:test_end_idx]
    

    #Define sampler and create TrainLoader and TestLoader
    sampler = torch.utils.data.SequentialSampler(arr)

    TrainLoader = torch.utils.data.DataLoader(train_data, batch_size=10, shuffle=False, sampler=sampler, pin_memory=True, drop_last=True)
    TestLoader = torch.utils.data.DataLoader(test_data, batch_size=10, shuffle=False, sampler=sampler, pin_memory=True, drop_last=True)
    return TrainLoader, TestLoader

In [13]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')
    
#train_on_gpu = False    

Training on GPU!


In [70]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_input = 7, n_output = 3, n_hidden = 64, n_layers=1,
                               drop_prob=0.2, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(n_input, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, n_output)
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''

        # Get the outputs and the new hidden state from the lstml
        print(x)
        r_output, hidden = self.lstm(x, hidden)
        print('Post LSTM Shape: ', r_output.shape)

        # Stack up LSTM outputs using view
        # You may need to use contiguous to reshape the output

        out = r_output.view(-1, self.n_hidden)
        print('Post Output Shape: ', out.shape)


        # Put x through the fully-connected layer
        out = self.dropout(self.fc(out))
        print('Post Drouput Shape: ', out.shape)    
                
        # Return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

In [71]:
def train(net, data, epochs=10, batch_size=10, seq_length=7, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    data, val_data = DataLoader(data, val_frac)

    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)

        for i, batch in enumerate(data):
            inputs = batch[:,:-1].float()
            inputs.unsqueeze_(1)   
            targets = batch[:,-1].int()
            targets.unsqueeze_(1) 
            targets = torch.tensor(one_hot_encode(targets, 3))
            counter += 1
         
            # Make our feature and target arrays Torch tensors
            print('input at counter position {}: {}'.format(counter, inputs.shape))

            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            # get the output from the model
            print('before net shape: {}'.format(inputs.shape))
            output, h = net(inputs, h)
            print('output is: {}'.format(output))
            print('after net shape: {}'.format(inputs.shape))

            # calculate the loss and perform backprop
            targets.squeeze_(-1)
            loss = criterion(output, targets.long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in enumerate(val_data):  
                    print(x)
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [72]:
#Import DataFrame from csv
arr = generate_train_test_data('chart_data-csv/aapl.csv').as_matrix()

# define and print the net
n_input=7
n_layers=1

net = CharRNN(arr, n_input, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(7, 64, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


C:\Users\jonat\Anaconda3\envs\zipline\lib\site-packages\torch\nn\modules\rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [73]:
batch_size = 10
seq_length = 7
n_epochs = 10 # start smaller if you are just testing initial behavior

# train the model
train(net, arr, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

input at counter position 1: torch.Size([10, 1, 7])
before net shape: torch.Size([10, 1, 7])
tensor([[[1.3473e+09, 8.6920e+01, 8.7570e+01, 8.5790e+01, 8.6330e+01,
          1.2600e+08, 9.8015e+07]],

        [[1.3474e+09, 8.7140e+01, 8.7540e+01, 8.5730e+01, 8.7530e+01,
          1.7806e+08, 9.9576e+07]],

        [[1.3475e+09, 8.8520e+01, 8.9580e+01, 8.8180e+01, 8.9250e+01,
          1.4959e+08, 1.0136e+08]],

        [[1.3476e+09, 9.0160e+01, 9.1080e+01, 8.9890e+01, 9.0340e+01,
          1.5012e+08, 1.0194e+08]],

        [[1.3479e+09, 9.1390e+01, 9.1450e+01, 9.0770e+01, 9.1450e+01,
          9.9508e+07, 1.0183e+08]],

        [[1.3480e+09, 9.1460e+01, 9.1780e+01, 9.1010e+01, 9.1730e+01,
          9.3376e+07, 1.0181e+08]],

        [[1.3480e+09, 9.1510e+01, 9.2000e+01, 9.1420e+01, 9.1750e+01,
          8.1719e+07, 1.0088e+08]],

        [[1.3481e+09, 9.1370e+01, 9.1480e+01, 9.0640e+01, 9.1310e+01,
          8.4142e+07, 1.0022e+08]],

        [[1.3482e+09, 9.1790e+01, 9.2140e+01, 9.139

RuntimeError: multi-target not supported at c:\a\w\1\s\tmp_conda_3.5_085150\conda\conda-bld\pytorch_1550393656615\work\aten\src\thcunn\generic/ClassNLLCriterion.cu:15

## OLD CODE

In [ ]:
# Define our network structure
class PricePredictNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Define our layer structure
        self.fc1 = nn.Linear(6, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 3)
        
        # Add dropout capability
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        
        # Flatten tensor
        x = x.view(x.shape[0], -1)
        
        # Feed forward
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))

        #Output layer -- no dropout
        x = F.log_softmax(self.fc3(x))
        
        return x

In [ ]:
# Define training/validation loop
model = PricePredictNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.02)

epochs = 250
train_losses = []
test_losses = []

# Train and Validate network
for e in range(epochs):
    train_loss = 0
    for sample in trainloader:
        # Get data and labels from sample
        data = sample['data']
        labels = torch.transpose(sample['label'],0,1)
        labels = labels.to(dtype=torch.int64)
        
        
        # Reset our gradiants
        optimizer.zero_grad()
        
        # Feed forward
        output = model(data)

        print(output)
        # Loss
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        # Tally up loss
        train_loss += loss.item()
    else:
        # Validation step
        model.eval()
        with torch.no_grad():
            test_loss = 0
            for sample in testloader:
                # Get data and labels from sample
                test_data = sample['data']
                labels = torch.transpose(sample['label'],0,1)
                
                # Run model
                output = model(test_data)
                output = torch.transpose(output,0,1)
                
                # Calculate loss
                loss = criterion(output, labels)
                test_loss += loss.item()
        
        # Record loss values
        train_losses.append(train_loss/len(trainset))
        test_losses.append(test_loss/len(testset))
        
        # Set model back to training mode to include dropout
        model.train()
    
    if(e%10 == 0):
        print('Epoch {0}\n-----------------'.format(str(e+1)))
        print('Training Loss: {0}'.format(train_losses[e]))
        print('Testing Loss: {0}\n'.format(test_losses[e]))

In [ ]:
# Plot performance of the training
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)

In [ ]:
# Network Stats
avg_train_loss = np.mean(train_losses)
avg_test_loss = np.mean(test_losses)
print('Average Training Loss: {0}'.format(avg_train_loss))
print('Average Testing Loss: {0}'.format(avg_test_loss))